## General Processing

Flow:
1. general filter: Exclude pediatric
2. outliers
3. train/test spliting
4. missing values: imputation




### Load master dataset

In [ ]:
import pandas as pd
import os
from helpers import *
path = 'C:/Users/XFE/Documents/mimic4ed-benchmark/data_processed/'
df_master = pd.read_csv(os.path.join(path, 'master_dataset.csv'))

In [ ]:
pd.set_option('display.max_columns', 100) 
df_master.head()

,index,subject_id,hadm_id,stay_id,intime,outtime,gender,dod,admittime,dischtime,deathtime,ethnicity,edregtime,edouttime,insurance,age,outcome_inhospital_mortality,ed_death,before_ed_mortality,ed_los,intime_icu,time_to_icu_transfer,outcome_icu_transfer_12h,n_ed_365d,next_ed_visit_time,next_ed_visit_time_diff,outcome_ed_revisit_3d,n_hosp_365d,n_icu_365d,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,acuity,chiefcomplaint,chiefcom_chest_pain,chiefcom_abdominal_pain,chiefcom_headache,chiefcom_shortness_of_breath,chiefcom_back_pain,chiefcom_cough,chiefcom_nausea_vomiting,chiefcom_fever_chills,chiefcom_syncope,chiefcom_dizziness
0,338038,10000032,22595853.0,33258284,2180-05-06 19:17:00,2180-05-06 23:30:00,F,NaN,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,Other,52,False,False,False,0 days 04:13:00,NaN,NaN,False,0,2180-06-26 15:54:00,50 days 16:24:00,False,0,0,98.4,70.0,16.0,97.0,106.0,63.0,0.0,3.0,"Abd pain, Abdominal distention",False,True,False,False,False,False,False,False,False,False
1,338039,10000032,22841357.0,38112554,2180-06-26 15:54:00,2180-06-26 21:31:00,F,NaN,2180-06-26 18:27:00,2180-06-27 18:49:00,NaN,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,Medicaid,52,False,False,False,0 days 05:37:00,NaN,NaN,False,1,2180-07-22 16:24:00,25 days 18:53:00,False,1,0,98.9,88.0,18.0,97.0,116.0,88.0,10.0,3.0,Abdominal distention,False,False,False,False,False,False,False,False,False,False
2,338040,10000032,29079034.0,32952584,2180-07-22 16:24:00,2180-07-23 05:54:00,F,NaN,2180-07-23 12:35:00,2180-07-25 17:55:00,NaN,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,Medicaid,52,False,False,False,0 days 13:30:00,2180-07-23 14:00:00,0 days 08:06:00,True,2,2180-07-23 05:54:00,0 days 00:00:00,True,2,0,97.8,87.0,14.0,97.0,71.0,43.0,7.0,2.0,Hypotension,False,False,False,False,False,False,False,False,False,False
3,338041,10000032,29079034.0,39399961,2180-07-23 05:54:00,2180-07-23 14:00:00,F,NaN,2180-07-23 12:35:00,2180-07-25 17:55:00,NaN,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,Medicaid,52,False,False,False,0 days 08:06:00,2180-07-23 14:00:00,0 days 00:00:00,True,3,2180-08-05 20:58:00,13 days 06:58:00,False,2,0,98.7,77.0,16.0,98.0,96.0,50.0,13.0,2.0,"Abdominal distention, Abd pain, LETHAGIC",False,True,False,False,False,False,False,False,False,False
4,338042,10000032,25742920.0,35968195,2180-08-05 20:58:00,2180-08-06 01:44:00,F,NaN,2180-08-05 23:44:00,2180-08-07 17:50:00,NaN,WHITE,2180-08-05 20:58:00,2180-08-06 01:44:00,Medicaid,52,False,False,False,0 days 04:46:00,NaN,NaN,False,4,NaN,NaN,False,3,1,99.4,105.0,18.0,96.0,106.0,57.0,10.0,3.0,"n/v/d, Abd pain",False,True,False,False,False,False,False,False,False,False


In [ ]:
#revise time range + function name

### 1. General filter - Age

In [ ]:
df_master = df_master[df_master['age'] >= 18]

### 2. Outlier Detection 

In [ ]:
# Mingrui add here?
# (first to plot/print the descriptive result to show how many outliers for each variable?)

In [ ]:
# from mimic-extract, to be revised
vitals_valid_range = {
    'temperature': {'outlier_low': 14.2, 'valid_low': 26., 'valid_high': 45, 'outlier_high':47.},
    'heartrate': {'outlier_low': 0, 'valid_low': 0, 'valid_high': 350, 'outlier_high':390},
    'resprate': {'outlier_low': 0, 'valid_low': 0., 'valid_high': 300., 'outlier_high':330.},
    'o2sat': {'outlier_low': 0, 'valid_low': 0, 'valid_high': 100, 'outlier_high':150},
    'sbp': {'outlier_low': 0, 'valid_low': 0., 'valid_high': 375, 'outlier_high':375.},
    'dbp': {'outlier_low': 0, 'valid_low': 0., 'valid_high': 375, 'outlier_high':375.},
    'pain': {'outlier_low': 0, 'valid_low': 0., 'valid_high': 10, 'outlier_high':10},
    'acuity': {'outlier_low': 1, 'valid_low': 1., 'valid_high': 5, 'outlier_high':5},
}

In [ ]:
df_master = convert_temp_to_celcius(df_master)

In [ ]:
display_outliers_count(df_master, vitals_valid_range)

In [ ]:
df_master = remove_outliers(df_master, vitals_valid_range)

### 3. Dataset Split (train:0.8, test: 0.2, use seed to fix)

In [ ]:
train=df_master.sample(frac=0.8,random_state=10) #random state is a seed value
test=df_master.drop(train.index)

In [ ]:
pd.set_option('display.max_columns', 100) 
train.head()

### 4. Missing Value imputation 

In [ ]:
# Mingrui add here?
# (first to plot/print the descriptive result to show how many missing valuesfor each variable?)
# use train to impute both train and test

### Output the train and test data

In [ ]:
df_master.to_csv(os.path.join(path, 'train.csv'), index=False)
df_master.to_csv(os.path.join(path, 'test.csv'), index=False)